In [28]:





#%%


# def read_secret(secret_name):
#     secret_path = os.getenv(secret_name)
#     try:
#         with open(secret_path, 'r') as file:
#             return file.read().strip()
#     except Exception as e:
#         print(f"Error reading {secret_name}: {e}")
#         return None

# INFLUXDB_USERNAME = read_secret('DOCKER_INFLUXDB_INIT_USERNAME_FILE')
# INFLUXDB_PASSWORD = read_secret('DOCKER_INFLUXDB_INIT_PASSWORD_FILE')
# INFLUXDB_TOKEN = read_secret('DOCKER_INFLUXDB_INIT_ADMIN_TOKEN_FILE')

# INFLUXDB_ORG = os.getenv('DOCKER_INFLUXDB_INIT_ORG')
# INFLUXDB_BUCKET = os.getenv('DOCKER_INFLUXDB_INIT_BUCKET')
# INFLUXDB_URL = os.getenv('INFLUXDB_URL', 'http://influxdb:8086') 
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.query_api import QueryOptions
import pandas as pd
import os
INFLUXDB_USERNAME="admin"
INFLUXDB_PASSWORD="password"
INFLUXDB_TOKEN="G3UtSut5Kv-RuT32yh27StdDCrl4fu3uzxPLzdias8vFsNzyzgfw5kIX9iGvtLctAXpZjFItOUwA65YWtk_5fg=="
INFLUXDB_ORG="example_org"
INFLUXDB_BUCKET="example_bucket"
INFLUXDB_URL="http://localhost:8086"

client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)

# Create a QueryAPI instance
query_api = client.query_api(query_options=QueryOptions(profilers=[]))





In [29]:
#%%
query = f'''
from(bucket: "{INFLUXDB_BUCKET}")
        |> range(start: -5h)
        |> filter(fn: (r) => r._measurement == "heart_rate")
        |> filter(fn: (r) => r._field == "noisy" or r._field == "original_time")
        |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        |> keep(columns: ["noisy", "original_time"])
'''
checkpoint_df = query_api.query_data_frame(query=query)
client.close()
checkpoint_df.drop(columns=['table','result'],inplace=True)
checkpoint_df['original_time'] = pd.to_datetime(checkpoint_df['original_time'])
latest_time = checkpoint_df['original_time'].max()
one_year_before_latest = latest_time - pd.DateOffset(years=1)
training_df = checkpoint_df[checkpoint_df['original_time'] >= one_year_before_latest]



# HERE TRAIN DATA USING TRAINING DF
# trainanomalymodel(training_df)

In [30]:
from datetime import datetime,timezone
import time

last_pull_time = datetime.utcnow().replace(tzinfo=timezone.utc) - pd.DateOffset(second=5)

print(last_pull_time.isoformat())
while True:
    # Build the query with dynamic start time
    query = f'''
    from(bucket: "{INFLUXDB_BUCKET}")
        |> range(start: {last_pull_time.isoformat()})
        |> filter(fn: (r) => r._measurement == "heart_rate")
        |> filter(fn: (r) => r._field == "noisy" or r._field == "original_time")
        |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
        |> keep(columns: ["noisy", "original_time"])
    '''
    new_data = query_api.query_data_frame(query=query)
    
    if not new_data.empty:
        new_data.drop(columns=['table', 'result'], inplace=True)
        new_data['original_time'] = pd.to_datetime(new_data['original_time'])
        
        # HERE TEST THIS NEW DATA FOR ANOMALY DETECTION.
        # detect_anomalies
        # ADD NON_ANOMALY ROWS TO ANOTHER DATAFRAME
        # IF ANOTHERDATAFRAME HAS 1 YEAR OF DATA, TRAIN DATA ON THAT.        
        #checkpoint_df = pd.concat([checkpoint_df, new_data])
        print(new_data)
        # Update the last pull time
        #last_pull_time = latest_time
        last_pull_time = datetime.utcnow().replace(tzinfo=timezone.utc)

    # Wait for 5 seconds before the next pull
    break
    time.sleep(5)

2024-09-15T10:02:05.307399+00:00
        noisy             original_time
0   63.564632 2022-06-13 18:00:00+00:00
1   63.018735 2022-06-14 09:00:00+00:00
2   63.673516 2022-06-14 12:00:00+00:00
3   63.469121 2022-06-14 15:00:00+00:00
4   63.836436 2022-06-14 18:00:00+00:00
5   61.043874 2022-06-15 09:00:00+00:00
6   60.598961 2022-06-15 12:00:00+00:00
7   60.869564 2022-06-15 15:00:00+00:00
8   60.705201 2022-06-15 18:00:00+00:00
9   61.193341 2022-06-16 09:00:00+00:00
10  61.037952 2022-06-16 12:00:00+00:00
11  61.067039 2022-06-16 15:00:00+00:00
12  60.969226 2022-06-16 18:00:00+00:00
13  57.462581 2022-06-17 09:00:00+00:00
14  57.635690 2022-06-17 12:00:00+00:00
15  57.939654 2022-06-17 15:00:00+00:00
16  57.995920 2022-06-17 18:00:00+00:00
17  59.530087 2022-06-18 09:00:00+00:00
18  59.437133 2022-06-18 12:00:00+00:00
19  59.359975 2022-06-18 15:00:00+00:00
20  60.244319 2022-06-18 18:00:00+00:00
21  57.581128 2022-06-19 09:00:00+00:00
22  58.253188 2022-06-19 12:00:00+00:00
23  58.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6573 entries, 0 to 6572
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   noisy          6564 non-null   float64            
 1   original_time  6573 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 102.8 KB


In [12]:
latest_time = result['original_time'].max()

In [13]:
latest_time

Timestamp('2021-08-09 12:00:00+0000', tz='UTC')